In [117]:
# lr:0.001
# anneal_rate:0.9
# batch_size:32
# clip_norm:50
# num_epochs:5
# alpha:250
# beta:0
# max_beta:1
# step_beta:0.002
# anneal_iter:40000
# kl_anneal_iter:2000
# print_iter:100
# save_iter:5000
# num_workers:4

In [118]:
!pip install -q torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric -f https://data.pyg.org/whl/torch-1.12.0+cu116.html
!pip install -q dive-into-graphs

In [119]:
!pip install -q toolz
!pip install -q wandb

In [139]:
%load_ext autoreload
%autoreload 2
import os
import json
import argparse
import pickle 

import numpy as np
import pandas as pd
import torch

from molecule_optimizer.externals.fast_jtnn.datautils import SemiMolTreeFolder, SemiMolTreeFolderTest
from molecule_optimizer.runner.semi_jtvae import SemiJTVAEGeneratorPredictor
from torch_geometric.data import DenseDataLoader

import rdkit

lg = rdkit.RDLogger.logger() 
lg.setLevel(rdkit.RDLogger.CRITICAL)

import warnings
warnings.filterwarnings("ignore")

import math

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [140]:
conf = json.load(open("training/configs/rand_gen_zinc250k_config_dict.json"))

In [141]:
csv = pd.read_csv("ZINC_310k.csv")

In [142]:
smiles = csv['SMILES']

In [165]:
smiles = smiles[:60000]

In [166]:
labels = torch.tensor(csv['QED'][:60000]).float()

In [167]:
# if 'runner.xml' not in os.listdir("."):
#     runner = SemiJTVAEGeneratorPredictor(smiles)
#     with open('runner.xml', 'wb') as f:
#         pickle.dump(runner, f)

In [168]:
if 'runner_20.xml' not in os.listdir("."):
    runner = SemiJTVAEGeneratorPredictor(smiles)
    with open('runner_20.xml', 'wb') as f:
        pickle.dump(runner, f)

In [169]:
# with open('runner.xml', 'rb') as f:
#     runner = pickle.load(f)

In [170]:
with open('runner_20.xml', 'rb') as f:
    runner = pickle.load(f)

In [171]:
runner.get_model(
    "rand_gen",
    {
        "hidden_size": conf["model"]["hidden_size"],
        "latent_size": conf["model"]["latent_size"],
        "depthT": conf["model"]["depthT"],
        "depthG": conf["model"]["depthG"],
        "label_size": 1,
        "label_mean": float(torch.mean(labels)),
        "label_var": float(torch.var(labels)),
    },
)

In [172]:
labels = runner.get_processed_labels(labels)
preprocessed = runner.processed_smiles

In [173]:
# N_TEST = 10000
N_TEST = 200
VAL_FRAC = 0.05

In [174]:
perm_id=np.random.permutation(len(labels))
X_train = preprocessed[perm_id[N_TEST:]]
L_train = torch.tensor(labels.numpy()[perm_id[N_TEST:]])


X_test = preprocessed[perm_id[:N_TEST]]
L_test = torch.tensor(labels.numpy()[perm_id[:N_TEST]])

In [175]:
val_cut = math.floor(len(X_train) * VAL_FRAC)

In [176]:
X_Val = X_train[:val_cut]
L_Val = L_train[:val_cut]

X_train = X_train[val_cut :]
L_train = L_train[val_cut :]

In [177]:
X_Val = X_train[:val_cut]
L_Val = L_train[:val_cut]

X_train = X_train[val_cut :]
L_train = L_train[val_cut :]

In [ ]:
print("Training model...")
runner.train_gen_pred(
    X_train,
    L_train,
    X_Val,
    L_Val,
    load_epoch=0,
    lr=conf["lr"],
    anneal_rate=conf["anneal_rate"],
    clip_norm=conf["clip_norm"],
    num_epochs=conf["num_epochs"],
    alpha=conf["alpha"],
    max_alpha=conf["max_alpha"],
    step_alpha=conf["step_alpha"],
    beta=conf["beta"],
    max_beta=conf["max_beta"],
    step_beta=conf["step_beta"],
    anneal_iter=conf["anneal_iter"],
    alpha_anneal_iter=conf["alpha_anneal_iter"],
    kl_anneal_iter=conf["kl_anneal_iter"],
    print_iter=100,
    save_iter=conf["save_iter"],
    batch_size=conf["batch_size"],
    num_workers=conf["num_workers"],
    label_pct=0.05
)

Training model...
Model #Params: 4732K
[Train][100] Alpha: 0.000, Beta: 0.000, Loss: 110.93, KL: 465.94, MAE: 0.57565, Word Loss: 83.27, Topo Loss: 19.08, Assm Loss: 8.58, Pred Loss: 19.24, Word: 33.20, Topo: 85.39, Assm: 57.83, PNorm: 102.05, GNorm: 50.00
[Train][200] Alpha: 0.000, Beta: 0.000, Loss: 68.44, KL: 453.67, MAE: 0.22728, Word Loss: 50.08, Topo Loss: 10.51, Assm Loss: 7.86, Pred Loss: 2.94, Word: 56.48, Topo: 92.75, Assm: 62.44, PNorm: 106.81, GNorm: 50.00
[Train][300] Alpha: 0.000, Beta: 0.000, Loss: 57.56, KL: 526.81, MAE: 0.18403, Word Loss: 41.57, Topo Loss: 8.67, Assm Loss: 7.32, Pred Loss: 2.05, Word: 64.32, Topo: 93.99, Assm: 64.57, PNorm: 110.55, GNorm: 50.00
[Train][400] Alpha: 0.000, Beta: 0.000, Loss: 51.08, KL: 612.44, MAE: 0.17472, Word Loss: 36.26, Topo Loss: 7.91, Assm Loss: 6.91, Pred Loss: 1.89, Word: 68.41, Topo: 94.48, Assm: 66.66, PNorm: 113.85, GNorm: 46.41
learning rate: 0.000729
[Train][500] Alpha: 0.000, Beta: 0.010, Loss: 48.14, KL: 380.73, MAE: 0.1

In [ ]:
print("Training model...")
runner.train_gen_pred_supervised(
    loader=train_loader,
    val_loader=val_loader,
    test_loader=test_loader,
    load_epoch=0,
    lr=conf["lr"],
    anneal_rate=conf["anneal_rate"],
    clip_norm=conf["clip_norm"],
    num_epochs=conf["num_epochs"],
    alpha=conf["alpha"],
    max_alpha=conf["max_alpha"],
    step_alpha=conf["step_alpha"],
    beta=conf["beta"],
    max_beta=conf["max_beta"],
    step_beta=conf["step_beta"],
    anneal_iter=conf["anneal_iter"],
    alpha_anneal_iter=conf["alpha_anneal_iter"],
    kl_anneal_iter=conf["kl_anneal_iter"],
    print_iter=100,
    save_iter=conf["save_iter"],
)